AINHOA GALLEGO - STAKED LSTM 60-64-64

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
#!pip install keras
from statsmodels.tsa.seasonal import seasonal_decompose
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
#!pip install keras --user
#!pip install tensorflow --user
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
#!pip install tensorflow --user
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten

In [10]:
df = pd.read_csv('ANA.MC2015.csv')

# Convertimos la columna Date en un objeto datetime
df['Date'] = pd.to_datetime(df['Date'])
# Agregamos columnas de año, mes y día
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.weekday  # Monday: 0, Sunday: 6 aunque no existen datos de fin de semana, la bolsa está cerrado; asi que es 0-4 de lunes-viernes

# Calcular el rendimiento diario
df['Return'] = df['Close'].pct_change()
df['Return'].fillna(0, inplace=True)

In [11]:
num_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Return']
# eliminamos los datos nulos
df = df.dropna()

In [14]:
# Definir las columnas que quieres representar
selected_cols = [col for col in num_cols if col != 'Return']

# Normalizar las columnas numéricas
scaler = MinMaxScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(df[selected_cols]), columns=selected_cols)

# Agregar la columna 'Date' al dataframe normalizado
scaled_df['Date'] = df['Date']

# Graficar las series temporales de las variables normalizadas
plt.figure(figsize=(14,8))

for column in selected_cols:
    plt.plot(scaled_df['Date'], scaled_df[column], label=column)

plt.xlabel('Fecha')
plt.ylabel('Valores Normalizados')
plt.title('Series Temporales de las Variables Normalizadas')
plt.legend()
plt.show()

In [15]:
#VAMOS A MOSTRAT ESTO
# Gráficas de las series temporales
plt.figure(figsize=(12,6))
plt.plot(df['Date'], df['Close'])
plt.title('Precio de cierre a lo largo del tiempo')
plt.xlabel('Fecha')
plt.ylabel('Precio de cierre')
plt.show()

## MODELO

In [16]:
# Seleccionamos las columnas 'Close' y 'Volume'
data = df[['Close', 'Volume']].values #ante ssolo hemso utilizado los datso pasados del valor de cierrer y ahroa hemos metido también el volumen. Podriamos meter todos pero como están tan correlacionados, creo que así es mejor

# Normalizamos los datos
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

scaler_close = MinMaxScaler()
scaler_close.fit(data[:, 0].reshape(-1, 1))

# Dividimos los datos en entrenamiento y prueba
train_size = int(len(scaled_data) * 0.6) #tambien cambiar
test_size = len(scaled_data) - train_size
train, test = scaled_data[0:train_size,:], scaled_data[train_size:len(scaled_data),:]

# Función para crear el dataset de entrenamiento
def create_dataset(dataset, look_back=1, look_forward=5):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-look_forward):
        a = dataset[i:(i+look_back), :]
        dataX.append(a)
        b = [dataset[i + j, 0] for j in range(look_back, look_back + look_forward)]
        dataY.append(b)
    return np.array(dataX), np.array(dataY)

# Creamos el dataset de entrenamiento
look_back = 60 #cuanto mira haci a atras #mismo
look_forward = 1 #mada, los mismos dias, 1
X_train, y_train = create_dataset(train, look_back, look_forward)

# Creamos el dataset de prueba
X_test, y_test = create_dataset(test, look_back, look_forward)

# Convertimos los datos a arreglos de numpy
X_train = np.reshape(X_train, (X_train.shape[0], look_back, X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], look_back, X_test.shape[2]))

## Modelo de RNN LSTM
# Definimos el modelo
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(look_forward)) #aqui me va a ir a ver cuantos dias estoy prediciendo hacia delante para poner ese numero de neuronas
model.compile(optimizer='adam', loss='mean_squared_error')
#vamos a tener más modelos proqie tenemos más capas y para cada una va a ir probando las diferentes alternativas

from keras.callbacks import EarlyStopping
# simple early stopping
es = EarlyStopping(monitor='loss', mode='min', verbose=1)

# Entrenamos el modelo
model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1, callbacks=[es]) #batch 16 y epoch 50

# Hacemos las predicciones
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invertimos las predicciones
train_predict = scaler_close.inverse_transform(train_predict)
y_train = scaler_close.inverse_transform(y_train)
test_predict = scaler_close.inverse_transform(test_predict)
y_test = scaler_close.inverse_transform(y_test)


Epoch 1/50
74/74 [==============================] - 8s 56ms/step - loss: 0.0013
Epoch 2/50
74/74 [==============================] - 4s 59ms/step - loss: 2.8414e-04
Epoch 2: early stopping
24/24 [==============================] - 1s 20ms/step


In [17]:
# Cálculo de métricas de error
def calc_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return [rmse, mae, r2]

metrics_train = calc_metrics(y_train, train_predict)
metrics_test = calc_metrics(y_test, test_predict)

metrics_df = pd.DataFrame(index=['RMSE', 'MAE', 'R2'], columns=['Train', 'Test'])
metrics_df['Train'] = metrics_train
metrics_df['Test'] = metrics_test

print(metrics_df)

         Train      Test
RMSE  2.378815  8.990538
MAE   1.832470  6.950680
R2    0.962283  0.925717


1/1 [==============================] - 0s 35ms/step


In [18]:
# Graficamos los datos
plt.figure(figsize=(15,8))

# Graficamos los datos originales
plt.plot(scaler_close.inverse_transform(scaled_data[:, 0].reshape(-1, 1)), color='black', label='Original data')

# Graficamos las predicciones de entrenamiento
plt.plot(range(look_back, len(train_predict) + look_back), 
         np.array([x[0] for x in train_predict]), color='blue', label='Train predictions')

# Graficamos las predicciones de prueba
plt.plot(range(len(train_predict) + look_back*2 + look_forward, len(scaled_data) - look_forward),
         np.array([x[0] for x in test_predict]), color='red', label='Test predictions')

# Graficamos las predicciones del próximo día
future_days = range(len(scaled_data) - look_forward, len(scaled_data))
future_predictions = model.predict(np.array([scaled_data[-look_back:]]))[0]
plt.scatter(future_days, scaler_close.inverse_transform(future_predictions.reshape(-1, 1)), 
            marker='x', color='green', label='Predicted next day')

plt.legend()
plt.show()

In [10]:
#TODAVIA ESTO NO LO CORRO
# Convertir los arrays de NumPy a DataFrames de pandas
real_data_df = pd.DataFrame(np.concatenate((y_train, y_test)), columns=['Real_Data'])

# Extender 'train_predict' con NaNs para que coincida con la longitud de 'real_data'
train_predict_extended = np.concatenate((train_predict, [np.nan]*len(y_test)))
train_predict_df = pd.DataFrame(train_predict_extended, columns=['Train_Fit'])

# Extender 'test_predict' con NaNs al principio para que coincida con la longitud de 'real_data'
test_predict_extended = np.concatenate(([np.nan]*len(y_train), test_predict))
test_predict_df = pd.DataFrame(test_predict_extended, columns=['Test_Predictions'])

# Concatenar estos DataFrames a lo largo del eje de las columnas
result_df = pd.concat([real_data_df, train_predict_df, test_predict_df], axis=1)

# Guardar el DataFrame resultante en un archivo .csv
result_df.to_csv('predictions.csv', index=False)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)